In [1]:
# import the necessary packages
import numpy as np
from imutils import face_utils
import dlib
import cv2
import math
import imutils

from locations import landmarks

In [2]:
# initialize dlib's face detector and create predictor
p = "shape_predictor_194_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(p)

In [3]:
#计算两向量的夹角
def calc_angle(v1, v2):
    dx1 = v1[2] - v1[0]
    dy1 = v1[3] - v1[1]
    dx2 = v2[2] - v2[0]
    dy2 = v2[3] - v2[1]
    angle1 = math.atan2(dy1, dx1)
    angle1 = int(angle1 * 180/math.pi)
    # print(angle1)
    angle2 = math.atan2(dy2, dx2)
    angle2 = int(angle2 * 180/math.pi)
    # print(angle2)
    if angle1*angle2 >= 0:
        included_angle = abs(angle1-angle2)
    else:
        included_angle = abs(angle1) + abs(angle2)
        if included_angle > 180:
            included_angle = 360 - included_angle
    return included_angle
'''
AB = [5,-1,1,-3]
CD = [4,1,4.5,4.5]
print(angle_new(AB, CD),type(AB))
'''

'\nAB = [5,-1,1,-3]\nCD = [4,1,4.5,4.5]\nprint(angle_new(AB, CD),type(AB))\n'

In [7]:
#calc features:
def feature_calc(video_path,n_frames=30):
    # Create video reader and find length
    v_cap = cv2.VideoCapture(video_path)
    v_len = int(v_cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # pick n frames
    sample = np.linspace(0, v_len - 1, n_frames).astype(int)

    # by frame index
    face_color_avgs = []
    face_color_stds = []
    yanpi_face_avgs = []
    lip_color_avgs = []
    center_base = []
    face_fluctuation = []
    brow_dists = []
    brow2nose_dists = []
    brow2eye_dists = []
    brow2nose_angles = []
    brow_thicks = []

    # part counter indexes
    r_eye_index = landmarks().get_polygons_index('R_EYE')
    l_eye_index = landmarks().get_polygons_index('L_EYE')
    xiaba_index = landmarks().get_polygons_index('XIABA')
    l_brow_index = landmarks().get_polygons_index('L_BROW')
    r_brow_index = landmarks().get_polygons_index('R_BROW')
    nose_index = landmarks().get_polygons_index('NOSE')
    u_lip_index = landmarks().get_polygons_index('U_LIP')
    d_lip_index = landmarks().get_polygons_index('D_LIP')
    
    for j in range(v_len):
        success = v_cap.grab()
        if j in sample:
            print('=========第{}帧========'.format(j))
            success, frame = v_cap.retrieve()
            if not success:
                continue

            frame = imutils.resize(frame, width = 1000)
            # detect faces in the grayscale image
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            rects = detector(gray, 0)
            if j == 0:
                face_cnt = len(rects)
                for _ in range(face_cnt):
                    face_color_avgs.append([])
                    face_color_stds.append([])
                    yanpi_face_avgs.append([])
                    lip_color_avgs.append([])
                    face_fluctuation.append([])
                    brow_dists.append([])
                    brow2nose_dists.append([])
                    brow2eye_dists.append([])
                    brow2nose_angles.append([[],[]])
                    brow_thicks.append([])


            for (i, rect) in enumerate(rects):
                shape = predictor(gray, rect)
                shape = face_utils.shape_to_np(shape)
                xia_lian = abs(shape[0][1] - shape[109][1])
                to_append_pnt = [[shape[0][0], int(shape[0][1] + xia_lian * 0.4)], 
                                   [shape[134][0], int(shape[134][1] + xia_lian * 0.4)]]



                # part counter points
                l_eye = shape[l_eye_index]
                r_eye = shape[r_eye_index]
                xiaba = np.concatenate((shape[xiaba_index], to_append_pnt), axis=0)
                u_lip = shape[u_lip_index]
                d_lip = shape[d_lip_index]
                l_brow = shape[l_brow_index]
                r_brow = shape[r_brow_index]
                nose = shape[nose_index]

                # make masks with coordinates
                mask_xiaba = np.zeros((frame.shape[0], frame.shape[1]))
                mask_eye = np.zeros((frame.shape[0], frame.shape[1]))
                mask_lip = np.zeros((frame.shape[0], frame.shape[1]))

                cv2.fillPoly(mask_xiaba, [xiaba], 1)
                cv2.fillPoly(mask_eye, [l_eye, r_eye], 1)
                cv2.fillPoly(mask_lip, [u_lip, d_lip], 1)

                mask_xiaba = mask_xiaba.astype(np.bool)
                xiaba_out = np.zeros_like(gray)
                xiaba_out[mask_xiaba] = gray[mask_xiaba]
                mask_eye = mask_eye.astype(np.bool)
                eye_out = np.zeros_like(gray)
                eye_out[mask_eye] = gray[mask_eye]
                mask_lip = mask_lip.astype(np.bool)
                lip_out = np.zeros_like(gray)
                lip_out[mask_lip] = gray[mask_lip]

                l_x, l_y, l_w, l_h = cv2.boundingRect(l_eye)
                l_eye_rectangle = gray[int(l_y-0.25*l_h):l_y+l_h, l_x:l_x+l_w]
                r_x, r_y, r_w, r_h = cv2.boundingRect(r_eye)
                r_eye_rectangle = gray[int(r_y-0.25*r_h):r_y+r_h, r_x:r_x+r_w]

                #distances
                center = [l_brow.mean(axis=0).tolist(),
                      r_brow.mean(axis=0).tolist(),
                      l_eye.mean(axis=0).tolist(),
                      r_eye.mean(axis=0).tolist(),
                      nose.mean(axis=0).tolist(),
                      np.concatenate((u_lip,d_lip)).mean(axis=0).tolist()]
                brow_dist = np.linalg.norm(shape[70] - shape[82])
                brow2nose_dist = np.linalg.norm(np.concatenate((shape[70],shape[82])).mean(axis=0) - shape[143])
                brow2eye_dist = (np.linalg.norm(np.array(center[0])-np.array(center[2])) + np.linalg.norm(np.array(center[1])-np.array(center[3]))) / 2
                brow_thick = (np.linalg.norm(shape[96] - shape[110]) + np.linalg.norm(shape[74] - shape[88])) / 2

                #angels
                nose_vec = np.array([shape[139],shape[147]]).mean(axis=0).tolist() + np.array([shape[135],shape[151]]).mean(axis=0).tolist()
                l_brow_vec = shape[103].tolist() + shape[113].tolist()
                r_brow_vec = shape[81].tolist() + shape[91].tolist()
                l_brow2nose_angel = calc_angle(nose_vec,l_brow_vec)
                r_brow2nose_angel = calc_angle(nose_vec,r_brow_vec)


                # calc feature
                face_color_avg = xiaba_out.sum() / mask_xiaba.sum()
                face_color_avgs[i].append(face_color_avg)
                face_color_stds[i].append(xiaba_out.std())
                yanpi_avg = (l_eye_rectangle.sum() + r_eye_rectangle.sum() - eye_out.sum()) / ((l_eye_rectangle.shape[0] + \
                        r_eye_rectangle.shape[0]) * (l_eye_rectangle.shape[1] + r_eye_rectangle.shape[1]) - mask_eye.sum())
                yanpi_face_avgs[i].append(yanpi_avg)
                lip_color_avg = lip_out.sum() / mask_lip.sum()
                lip_color_avgs[i].append(lip_color_avg)
                if j == 0:
                    center_base.append(center)
                else:
                    face_fluctuation[i].append(np.linalg.norm(np.array(center)-np.array(center_base[i]),axis=1).std())
                    center_base[i] = center.copy()
                brow_dists[i].append(brow_dist)
                brow2nose_dists[i].append(brow2nose_dist)
                brow2eye_dists[i].append(brow2eye_dist)
                brow2nose_angles[i][0].append(l_brow2nose_angel)
                brow2nose_angles[i][1].append(r_brow2nose_angel)
                brow_thicks[i].append(brow_thick)


    v_cap.release()
    
    feature_face_color = [[np.array(face_color_avgs[i]).std()] for i in range(face_cnt)]# 面部色度标准差
    feature_face_maxdiff = [[max(face_color_stds[i])] for i in range(face_cnt)] # 面部色度最大差异（最大标准差）
    feature_yanpi_face_diff = [[np.array(yanpi_face_avgs[i]).std()] for i in range(face_cnt)] # 眼皮面部色度差的标准差
    feature_lip_color = [[np.array(lip_color_avgs[i]).std()] for i in range(face_cnt)]#嘴唇色度标准差
    feature_face_fluctuation = [[max(face_fluctuation[i])] for i in range(face_cnt)]#面部特征移动相对距离
    feature_brow_dist = [[np.array(brow_dists[i]).std()] for i in range(face_cnt)]#眉毛距离标准差
    feature_brow2nose_dist = [[np.array(brow2nose_dists[i]).std()] for i in range(face_cnt)]#眉心到鼻尖距离标准差
    feature_brow2eye_dist = [[np.array(brow2eye_dists[i]).std()] for i in range(face_cnt)]#眉毛到眼睛距离标准差
    feature_brow2nose_angle = [[np.array(brow2nose_angles[i][0]).std(),np.array(brow2nose_angles[i][1]).std()] 
                               for i in range(face_cnt)]#眉毛与鼻子夹角，左右眉毛都做了计算
    feature_brow_thick = [[np.array(brow_thicks[i]).std()] for i in range(face_cnt)]#眉毛粗细


    feature = []
    for i in range(face_cnt):
        feature.append(feature_face_color[i] +
                       feature_face_maxdiff[i] +
                       feature_yanpi_face_diff[i] +
                       feature_lip_color[i] +
                       feature_face_fluctuation[i] +
                      feature_brow_dist[i] +
                      feature_brow2nose_dist[i] +
                      feature_brow2eye_dist[i] +
                      feature_brow2nose_angle[i] +
                      feature_brow_thick[i])

    return feature

In [8]:
def main():
    
    return feature_calc('../data/aczrgyricp.mp4')

In [38]:
import pandas as pd
import json

metadata = json.loads(open('../data/metadata.json').read())
features = []
for video in metadata.keys():
    video_path = os.path.join('../data',video)
    if os.path.exists(video_path):
            if metadata[video]['label'] == 'FAKE':
                label = 0
            else:
                label = 1
                feature = feature_calc(video_path)
                for i in range(len(feature)):
                    features.append(feature[i].append(label).append(video))
                    
res = pd.DataFrame(features,columns = ['face_color',
                                   'face_maxdiff',
                                   'yanpi_face_diff',
                                   'lip_color',
                                   'face_fluctuation',
                                   'brow_dist','brow2nose_dist',
                                   'brow2eye_dist',
                                   'l_brow2nose_angle',
                                   'r_brow2nose_angle',
                                   'brow_thick',
                                   'label','video'])
                
        